# Regression Model B: Field 5

## Install libraries

In [1]:
install.packages('readxl')
install.packages('corrplot')
install.packages('mice')
install.packages('RColorBrewer')
install.packages('car')
install.packages('Metrics')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("mice"):
“installation of package ‘mice’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


## Call libraries

In [2]:
library(readxl)
library(corrplot)
library(mice)
library(RColorBrewer)
library(car)
library(Metrics)

corrplot 0.84 loaded
Loading required package: lattice
Loading required package: carData


## Import data

In [3]:
d1=read_excel("variable2.xlsx",sheet = 1)
d2=read_excel("variable2.xlsx",sheet = 2)
d3=read_excel("variable2.xlsx",sheet = 3) 
d4=read_excel("variable2.xlsx",sheet = 4)
d5=read_excel("variable2.xlsx",sheet = 5) 

## Field 5 (F5)

Call the data according to the Field. 

In [4]:
dE1=d1[which(d1$`ESTATE CODE`=="E5" ),]
dE2=d2[(d2$Diff ),]
dE3=d3[which(d3$Estate=="E5" ),]
dE4=d4[which(d4$Estate=="E5" ),]
dE5=d5[which(d5$Estate=="E5" ),]

## Variables

Declare the variables. The variables in which pruning time, pruning cost, manuring time, manusring cost, P&D time and P&D cost contains zero values. In this case, we assume that the data is not available (NA). Then, we construct a new table for easy representation. 

In [5]:
year=dE1$`CALENDER YEAR`
month=dE1$`CALENDER MONTH`
yield=dE1$`CPB TONNAGE`
rainfall=dE1$RAINFALL

foliar=d2$Diff

prun.time=dE3$Frequency
prun.time[prun.time == 0] <- NA

prun.cost=dE3$PruningCost
prun.cost[prun.cost == 0] <- NA

manuring.time=dE4$Frequency
manuring.time[manuring.time == 0] <- NA

manuring.cost=dE4$ManuringCost
manuring.cost[manuring.cost == 0] <- NA

pd.time=dE5$Frequency
pd.time[pd.time == 0] <- NA

pd.cost=dE5$PDCost
pd.cost[pd.cost == 0] <- NA


df1=data.frame(year,month,yield,rainfall,foliar,prun.time,prun.cost,
               manuring.time,manuring.cost,pd.time,pd.cost)

## Identify missing values in each variables

In this case, we impute the missing values. Imputation is the process of replacing missing data with substituted values.  

This study use MICE (Multivariate Imputation via Chained Equations). MICE (Multivariate Imputation via Chained Equations) is one of the commonly used package by R users. Creating multiple imputations as compared to a single imputation (such as mean) takes care of uncertainty in missing values.

MICE assumes that the missing data are Missing at Random (MAR), which means that the probability that a value is missing depends only on observed value and can be predicted using them. It imputes data on a variable by variable basis by specifying an imputation model per variable.


The mice package has function known as md.pattern(). It returns a tabular for of missing value present in each variable in a data set.

In [6]:
md.pattern(df1)

,year,month,yield,rainfall,foliar,pd.time,pd.cost,manuring.time,manuring.cost,prun.time,prun.cost,
10,1,1,1,1,1,1,1,1,1,1,1,0
14,1,1,1,1,1,1,1,1,1,0,0,2
6,1,1,1,1,1,1,1,0,0,1,1,2
2,1,1,1,1,1,1,1,1,0,0,0,3
6,1,1,1,1,1,1,1,0,0,0,0,4
4,1,1,1,1,1,0,0,1,1,0,0,4
2,1,1,1,1,1,0,0,0,0,0,0,6
,0,0,0,0,0,6,6,14,16,28,28,98


## Imputation

Here is an explanation of the parameters used:  
m  – Refers to 5 imputed data sets.  
maxit – Refers to no. of iterations taken to impute missing values.  
method – Refers to method used in imputation. We used predictive mean matching.

In [7]:
imputed_Data <- mice(df1[,-1:-4], m=5, maxit = 50, method = 'pmm', seed = 500)
summary(imputed_Data)


 iter imp variable
  1   1  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   2  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   3  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   4  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   5  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   1  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   2  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   3  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   4  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   5  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  3   1  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  3   2  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  3   3  prun.time  prun.cost  manuring.time

Multiply imputed data set
Call:
mice(data = df1[, -1:-4], m = 5, method = "pmm", maxit = 50, 
    seed = 500)
Number of multiple imputations:  5
Missing cells per column:
       foliar     prun.time     prun.cost manuring.time manuring.cost 
            0            28            28            14            16 
      pd.time       pd.cost 
            6             6 
Imputation methods:
       foliar     prun.time     prun.cost manuring.time manuring.cost 
        "pmm"         "pmm"         "pmm"         "pmm"         "pmm" 
      pd.time       pd.cost 
        "pmm"         "pmm" 
VisitSequence:
    prun.time     prun.cost manuring.time manuring.cost       pd.time 
            2             3             4             5             6 
      pd.cost 
            7 
PredictorMatrix:
              foliar prun.time prun.cost manuring.time manuring.cost pd.time
foliar             0         0         0             0             0       0
prun.time          1         0         1           

Since there are 5 imputed data sets, we select any using complete() function.
Get complete data (2nd out of 5)

Get the complete data after impute the missing value.

In [8]:
completeData <- complete(imputed_Data,2)
df2=data.frame(df1[,1:4],completeData)

## Correlation

Check the correlations with all variables. 

In [9]:
r1=cor(df2[,-1:-2])

corrplot(r1, method="number",type="upper",col=brewer.pal(n=8, name="RdYlBu"))
corrplot(r1, type="upper")

## Train and test data

Split the dataset into train and test sets. After a model has been processed by using the training set, we test the model by making predictions against the test set. 

Because the data in the testing set already contains known values for the attribute that we want to predict, it is easy to determine whether the model's guesses are correct.

In this case, the datasets from May 2014 until December 2016 is considered for training dataset. The test dataset is from January 2017 until December 2017. 


In [10]:
train = df2[df2$year<=2016,]
test = df2[df2$year==2017,]
dim(train)
dim(test)

[1] 32 11

[1] 12 11

## Regression

Fit the data using multiple regression for all variables. 

In [11]:
k1=lm(yield~rainfall+foliar+prun.time+prun.cost+
        manuring.time+manuring.cost+pd.time+pd.cost,data=train)
summary(k1)
plot(k1,which=1)
plot(k1,which=2)


Call:
lm(formula = yield ~ rainfall + foliar + prun.time + prun.cost + 
    manuring.time + manuring.cost + pd.time + pd.cost, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-55.265  -9.618   0.248  19.790  40.188 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)  
(Intercept)    1.225e+02  5.006e+01   2.448   0.0224 *
rainfall       4.140e-02  5.456e-02   0.759   0.4557  
foliar        -3.066e+00  2.121e+00  -1.445   0.1618  
prun.time      2.873e+00  1.387e+01   0.207   0.8377  
prun.cost      5.783e-03  6.064e-03   0.954   0.3502  
manuring.time  1.026e+01  1.039e+01   0.988   0.3333  
manuring.cost -2.215e-04  3.431e-04  -0.646   0.5249  
pd.time       -1.335e+01  9.991e+00  -1.337   0.1944  
pd.cost        1.007e-03  6.623e-03   0.152   0.8805  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 28.88 on 23 degrees of freedom
Multiple R-squared:  0.4381,	Adjusted R-squared:  0.2427 
F-statistic: 2.24

## Significant variable 

Check the significant variables using two ways. 

a.	anova  
Choose the variables that has significant value in which p<0.1.

b.	using stepwise method  
In order to avoid the multicollinearity issues in variables selection, we take into account the variance inflation factors (VIF). We remove the variables that has VIF greater than 4. Then we rebuild the model. 

The stepwise method incorporate with VIF factor is better than anova method in order to select the significant variables. 

### Using ANOVA

In [12]:
anova(k1)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
rainfall,1,133.07828,133.07828,0.15951502,0.69328795
foliar,1,9526.32196,9526.32196,11.41877840,0.00258573
prun.time,1,491.40497,491.40497,0.58902528,0.45060385
prun.cost,1,664.72213,664.72213,0.79677285,0.38130451
manuring.time,1,623.33883,623.33883,0.74716853,0.39629995
manuring.cost,1,1167.56293,1167.56293,1.39950575,0.24889101
pd.time,1,2335.11838,2335.11838,2.79900254,0.10787207
pd.cost,1,19.26927,19.26927,0.02309721,0.88052981
Residuals,23,19188.16509,834.26805,NA,NA


### Using stepwise

In [13]:
selectedMod <- step(k1)
summary(selectedMod)

##Avoid Multicollinearity and Statistical Significance
##Recursively remove variables with variance inflation factors: VIF > 4
# Remove vars with VIF> 4 and re-build model until none of VIFs don't exceed 4.
all_vifs <- car::vif(selectedMod)
print(all_vifs)

while(any(all_vifs > 4)){
  var_with_max_vif <- names(which(all_vifs == max(all_vifs)))  # get the var with max vif
  signif_all <- signif_all[!(signif_all) %in% var_with_max_vif]  # remove
  myForm <- as.formula(paste("ozone_reading ~ ", paste (signif_all, collapse=" + "), sep=""))  # new formula
  selectedMod <- lm(myForm, data=inputData)  # re-build model with new formula
  all_vifs <- car::vif(selectedMod)
}

summary(selectedMod)

Start:  AIC=222.68
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- pd.cost        1     19.27 19207 220.71
- prun.time      1     35.83 19224 220.74
- manuring.cost  1    347.82 19536 221.26
- rainfall       1    480.27 19668 221.47
- prun.cost      1    758.77 19947 221.92
- manuring.time  1    814.71 20003 222.01
<none>                       19188 222.68
- pd.time        1   1490.49 20679 223.08
- foliar         1   1743.03 20931 223.46

Step:  AIC=220.71
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.time

                Df Sum of Sq   RSS    AIC
- prun.time      1     80.96 19288 218.85
- manuring.cost  1    408.73 19616 219.39
- rainfall       1    477.56 19685 219.50
- prun.cost      1    739.52 19947 219.92
- manuring.time  1    980.04 20188 220.31
<none>                       19207 220.71
- foliar         1   1725.76 20933 221.47



Call:
lm(formula = yield ~ foliar + pd.time, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-63.514 -10.322  -0.629  21.681  41.546 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  167.693     13.720  12.222 5.79e-13 ***
foliar        -4.610      1.476  -3.123  0.00404 ** 
pd.time      -13.884      6.627  -2.095  0.04500 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 27.31 on 29 degrees of freedom
Multiple R-squared:  0.3664,	Adjusted R-squared:  0.3227 
F-statistic: 8.386 on 2 and 29 DF,  p-value: 0.001337


  foliar  pd.time 
1.028261 1.028261 



Call:
lm(formula = yield ~ foliar + pd.time, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-63.514 -10.322  -0.629  21.681  41.546 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  167.693     13.720  12.222 5.79e-13 ***
foliar        -4.610      1.476  -3.123  0.00404 ** 
pd.time      -13.884      6.627  -2.095  0.04500 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 27.31 on 29 degrees of freedom
Multiple R-squared:  0.3664,	Adjusted R-squared:  0.3227 
F-statistic: 8.386 on 2 and 29 DF,  p-value: 0.001337


## Prediction

The performance of the model is evaluated by comparing the estimated value with the actual value. In this case, the test dataset (2017 datasets) is used to examine the performance of model with significant variables. 

Construct the table and graph that represents the estimated and observed value. 

In [14]:
estimate=predict(selectedMod,newdata=test)
compare=data.frame(test$yield,estimate)
names(compare)=c("Actual","Estimate")
compare

plot(compare$Actual,type="b",col="blue",xlab="Actual",ylab="Predicted",pch=19)
lines(fitted(selectedMod),col="red",type="b",pch=18)
legend("topleft",legend=c("Actual","Predicted"),col=c("blue","red"),lty=1,cex=0.8)

,Actual,Estimate
,<dbl>,<dbl>
33,137.894,89.15879
34,90.749,98.43276
35,63.475,79.93835
36,64.678,89.21232
37,83.421,153.80894
38,103.095,135.31453
39,140.657,130.70431
40,145.893,126.09409
41,160.711,135.36806


## Root mean square error (RMSE)

Calculate the root mean square error (RMSE) of the model. Extract the coefficient of determination (  ) to know how close the data to the fitted regression line. 

In [15]:
rmse=rmse(test$yield,estimate )
fit=c(RMSE = rmse, R2=summary(k1)$r.squared)
fit

RMSE         R2 
38.3020016  0.4381043